ref :

https://colab.research.google.com/drive/1Ucu0BHXFTp70jyqNOMUwX_FLEhyE8kJv?usp=sharing%EF%BB%BF#scrollTo=FSTi00anFGqf

https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression/


In [1]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# retriever 구성 후, llm으로 해당 내용 요약
# 요약된 내용을 다시 vector db에 저장
# 이후 user input에 대해서는 요약내용이 포함된 vector db를 바탕으로 retriever 동작 및 llm 답변 진행

# 데이터셋 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=580
# 샘플데이터 > 판결문 > 원천데이터 > 2021~~ 중 4개 데이터로 테스트

from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./dataset/", glob="*.xml", show_progress=True)
docs = loader.load()

  0%|          | 0/4 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


In [3]:
len(docs)

4

In [4]:
docs[0]

Document(page_content='\n\t\t\n211103_행정사건PDF\t\t\n\t\t\n\n\n\t\t\n2021누10599.pdf\t\t\n\t\t\n\n\n\t\t\n\n, 1 ,\n\n광  주  고  등  법  원\n\n제 2 - 1 행 정 부\n\n판             결\n\n사       건 2021누10599  상수도원인자부담금 부과처분 무효확인\n\n원고, 피항소인 000 주식회사\n\n소송대리인 법무법인 000\n\n           담당변호사 00\n\n피고, 항소인 순천시맑은물관리센터소장\n\n제 1심판결 광주지방법원 2021. 2. 4. 선고 2020구합13141 판결\n\n변 론 종 결 2021. 6. 9.\n\n판 결 선 고 2021. 7. 7.\n\n주       문\n\n1. 피고의 항소를 기각한다.\n\n2. 항소비용은 피고가 부담한다.\n\n청구취지 및 항소취지\n\n1. 청구취지\n\n  피고가 2016. 11. 17. 원고에 대하여 한 순천시 B택지지구 공동주택용지 C 블록 세\n\n\n\n, 2 ,\n\n대분 상수도원인자부담금 75,600,000원 및 위 지역 상가분 상수도원인자부담금 \n\n2,410,000원의 각 부과처분은 무효임을 확인한다.\n\n2. 항소취지\n\n  제1심판결을 취소한다. 원고의 청구를 모두 기각한다.\n\n이       유\n\n1. 제1심판결의 인용\n\n  이 법원이 이 사건에 관하여 설시할 이유는 제1심판결문 7면 4행과 5행 사이에 아\n\n래 \uf000 \uf000 부분을 추가하는 것 외에는 제1심판결의 이유 기재와 같으므로, 행정소송법 제\n\n8조 제2항, 민사소송법 제420조 본문에 따라 이를 그대로 인용한다(피고가 항소하면서 \n\n이 법원에서 주장하는 사유는 제1심에서 피고가 주장한 내용과 크게 다르지 아니하고, \n\n제1심과 이 법원에서 제출된 증거들을 모두 살펴보더라도 피고의 주장을 배척한 제1심\n\n의 사실인정과 판단은 정당하다).\n\n\uf000

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# 한글 데이터 임베딩 모델
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embedding_model = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# vector store 구성
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model)

In [7]:
retriever = vectorstore.as_retriever()

In [11]:
# retriever test

retrieved_docs = retriever.invoke("대전고등법원과 관련된 판례")

In [12]:
print(retrieved_docs[0])

page_content='\n\t\t\n211103_행정사건PDF\t\t\n\t\t\n\n\n\t\t\n2021누10574.pdf\t\t\n\t\t\n\n\n\t\t\n\n, 1 ,\n\n대  전  고  등  법  원\n\n제 2 행 정 부\n\n판           결\n\n사       건 2021누10574  원인자부담금 부과처분 취소 청구의 소\n\n원고, 피항소인 한국토지주택공사\n\n소송대리인 법무법인 000, 담당변호사 00\n\n피고, 항소인 대전광역시 상수도사업본부 유성사업소장\n\n소송대리인 법무법인 000, 담당변호사 00\n\n제 1심판결 대전지방법원 2021. 2. 3. 선고 2019구합107257 판결\n\n변 론 종 결 2021. 7. 2.\n\n판 결 선 고 2021. 8. 20.\n\n주       문\n\n1. 피고의 항소를 기각한다.\n\n2. 항소비용은 피고가 부담한다.\n\n청구취지 및 항소취지\n\n1. 청구취지\n\n  피고가 2019. 7. 16. 원고에게 0000에 관하여 한 92,344,000\n\n\n\n, 2 ,\n\n원의 원인자부담금 부과처분을 취소한다.  \n\n2. 항소취지\n\n  제1심 판결을 취소한다. 원고의 청구를 기각한다.\n\n이       유\n\n1. 처분의 경위\n\n  이 법원이 이 부분에 적을 이유는 원심판결 해당 부분 기재와 같으므로, 행정소송법 \n\n제8조 제2항, 민사소송법 제420조 본문에 의하여 이를 그대로 인용한다.   \n\n2. 이 사건 처분의 적법 여부\n\n  가. 원고의 주장1)\n\n  원고는 이 사건 각 사업지구 내 상수도시설의 설치비용을 직접 부담하여 실질적으로 \n\n상수도 원인자부담금 납부의무를 이행하였으므로 수도법 제71조에 따른 원인자부담금\n\n의 부과사유가 존재하지 않는다. \n\n  나. 관련 법령\n\n  별지와 같다.\n\n  다. 판단\n\n    1) 관련 법리\n\n      가) 부담금관리법 제5조 제1항은 부담금은 설치목적을 달성하기 

In [13]:
!pip install python-dotenv

In [14]:
# gpt 이용해서 판결문 요약
from dotenv import load_dotenv
import os

load_dotenv('.env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [15]:
# 긴 context 다루기 위해 gpt-3.5-turbo-16k 이용
# 요약하기 위함

from langchain.chat_models import ChatOpenAI

# 이상한 요약 방지를 위해 temperature = 0 으로 설정
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0, openai_api_key=OPENAI_API_KEY)

In [16]:
from langchain.prompts.chat import ChatPromptTemplate

template = """
당신은 판례 요약에 대해 전문가이다.
판례가 주어지면 주요 내용을 요약한다.
사건 번호, 판결 선고일, 형량, 범죄사실은 반드시 포함한다.
"""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
  ("system", template),
  ("human", human_template),
])

In [17]:
chain = chat_prompt | llm

In [18]:
# 요약 테스트
test_result = chain.invoke({"text": docs[0]})

test_result

AIMessage(content='사건 번호: 2021누10599\n판결 선고일: 2021. 7. 7.\n형량: 없음\n범죄사실: 상수도원인자부담금 부과처분 무효확인\n\n판결 요지:\n- 피고의 항소를 기각한다.\n- 항소비용은 피고가 부담한다.\n- 원고의 청구인 상수도원인자부담금 부과처분은 무효로 확인된다.\n- 제1심판결을 취소하고 원고의 청구를 모두 기각한다.\n\n이유:\n- 피고의 주장은 제1심에서 주장한 내용과 크게 다르지 않으며, 제1심의 사실인정과 판단은 정당하다.\n- 상수도원인자부담금 부과처분은 적법하고 유효하다고 주장하는 피고의 주장은 타당하지 않다.\n- 원고가 특별한 사정 없이 건축물을 건축한 자에게 상수도원인자부담금을 부담하게 할 수는 없다.\n- 따라서 제1심판결은 정당하며, 피고의 항소는 이유 없어 기각된다.', response_metadata={'token_usage': {'completion_tokens': 377, 'prompt_tokens': 2158, 'total_tokens': 2535}, 'model_name': 'gpt-3.5-turbo-16k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6767ff6e-ec09-4d8b-966a-5d4d8cc0ebd6-0')

In [19]:
from langchain.docstore.document import Document

summary_documents = []

# 최대 context 16k 이므로 내용 이것보다 많으면 자르기
max_len = 16000
for idx, doc in enumerate(docs) :
  if len(doc.page_content) > max_len :
    doc.page_content = doc.page_content[:max_len]
  result = chain.invoke({"text": doc})
  page = Document(page_content = result.content)
  summary_documents.append(page)

In [20]:
print(summary_documents[0])

page_content='사건 번호: 2021누10599\n판결 선고일: 2021. 7. 7.\n형량: 없음\n범죄사실: 상수도원인자부담금 부과처분 무효확인\n\n판결 요지:\n- 피고의 항소를 기각한다.\n- 항소비용은 피고가 부담한다.\n- 원고의 청구인 상수도원인자부담금 부과처분은 무효로 확인된다.\n- 제1심판결을 취소하고 원고의 청구를 모두 기각한다.\n\n이유:\n- 피고의 주장은 제1심에서 주장한 내용과 크게 다르지 않으며, 제1심의 사실인정과 판단은 정당하다.\n- 상수도원인자부담금 부과처분은 적법하고 유효하다고 주장하는 피고의 주장은 타당하지 않다.\n- 원고가 특별한 사정 없이 건축물을 건축한 자에게 상수도원인자부담금을 부담하게 할 수는 없다.\n- 따라서 제1심판결은 정당하며, 피고의 항소는 이유 없어 기각된다.'


In [21]:
# 요약된 document를 vector db에 저장

summary_vectorstore = Chroma.from_documents(documents=summary_documents, embedding=embedding_model, persist_directory='./dataset/')

In [22]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [23]:
summary_retriever = summary_vectorstore.as_retriever()

In [28]:
summary_search_results = summary_retriever.get_relevant_documents("학교용지부담금 관련된 판례를 알려줘")
pretty_print_docs(summary_search_results)

Document 1:

사건 번호: 2021누20580
판결 선고일: 2021.5.28
형량: 판결 내용에는 형량에 대한 언급이 없음
범죄사실: 개발부담금 부과처분 취소를 구하는 소송. 원고는 기부채납시설 가액을 개발비용에 포함하여 개발부담금을 산정해야 한다고 주장하였으나, 피고는 이를 공제하지 않았다. 법원은 피고의 처분이 적법하다고 판결하였다.
----------------------------------------------------------------------------------------------------
Document 2:

사건 번호: 2021누20269
판결 선고일: 2021.7.21.
형량: 원고의 항소 기각, 항소비용은 원고가 부담
범죄사실: 학교용지부담금 부과처분 취소

이 사건은 A지역주택조합이 부산진구청장에 대한 학교용지부담금 부과처분을 취소하기 위해 항소한 사건이다. 원고의 항소는 기각되었고, 항소비용은 원고가 부담하게 되었다. 판결은 2020년 12월 17일에 선고된 제1심판결을 취소하고, 피고가 원고에 대해 부과한 2,412,167,520원의 학교용지부담금 부과처분을 취소하는 내용이다.

판결에서는 원고의 주장이 제1심판결과 크게 다르지 않으며, 제1심판결의 사실인정과 판단이 정당하다고 판단되었다. 또한, 구 학교용지법은 학교용지의 확보를 쉽게 하기 위해 학교용지부담금을 부과하는 것을 목적으로 하고 있으며, 이 사건 개발사업에 대해서 전체 세대수를 기준으로 학교용지부담금을 부과한 것이 목적에 위반되는 것으로 단정하기 어렵다고 판시하였다. 또한, 개발사업에 대한 제한 규정은 구 학교용지법 제5조 제1항 단서에서 명확히 규정되어 있으며, 이를 재량권을 일탈한 것으로 보기 어렵다고 판시하였다.

따라서, 원고의 항소는 이유 없이 기각되었고, 제1심판결이 정당하다고 판결되었다.
------------------------------------------------------------------------------------

In [29]:
# metadata 검색을 통한 정확한 판례 찾기

def find_exact_case(case_number):
    for doc in docs:
      if doc.metadata["source"].find(case_number) != -1:
          return doc.page_content

In [30]:
print(find_exact_case("2021누10599"))


		
211103_행정사건PDF		
		


		
2021누10599.pdf		
		


		

, 1 ,

광  주  고  등  법  원

제 2 - 1 행 정 부

판             결

사       건 2021누10599  상수도원인자부담금 부과처분 무효확인

원고, 피항소인 000 주식회사

소송대리인 법무법인 000

           담당변호사 00

피고, 항소인 순천시맑은물관리센터소장

제 1심판결 광주지방법원 2021. 2. 4. 선고 2020구합13141 판결

변 론 종 결 2021. 6. 9.

판 결 선 고 2021. 7. 7.

주       문

1. 피고의 항소를 기각한다.

2. 항소비용은 피고가 부담한다.

청구취지 및 항소취지

1. 청구취지

  피고가 2016. 11. 17. 원고에 대하여 한 순천시 B택지지구 공동주택용지 C 블록 세



, 2 ,

대분 상수도원인자부담금 75,600,000원 및 위 지역 상가분 상수도원인자부담금 

2,410,000원의 각 부과처분은 무효임을 확인한다.

2. 항소취지

  제1심판결을 취소한다. 원고의 청구를 모두 기각한다.

이       유

1. 제1심판결의 인용

  이 법원이 이 사건에 관하여 설시할 이유는 제1심판결문 7면 4행과 5행 사이에 아

래   부분을 추가하는 것 외에는 제1심판결의 이유 기재와 같으므로, 행정소송법 제

8조 제2항, 민사소송법 제420조 본문에 따라 이를 그대로 인용한다(피고가 항소하면서 

이 법원에서 주장하는 사유는 제1심에서 피고가 주장한 내용과 크게 다르지 아니하고, 

제1심과 이 법원에서 제출된 증거들을 모두 살펴보더라도 피고의 주장을 배척한 제1심

의 사실인정과 판단은 정당하다).

 4) 피고는 순천시 상수도원인자부담금 산정․징수 등에 관한 조례 제4조 제1항 

제3호에서 급수구역 내에 위치하는 건축물 등에 수돗물을 공급하는 경우에 기존에 소

요된 수도시설의 건설비를 수돗물을 사용할 자에게 부담시키는 경우